# **INSTALLING THE NECESSARY LIBRARIES**

In [4]:
pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.3 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip -q install datasets

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install transformers[torch]

Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install --upgrade transformers

Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install accelerate -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 2.5 MB/s eta 0:00:0000:0100:01m
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.28.0
    Uninstalling accelerate-0.28.0:
      Successfully uninstalled accelerate-0.28.0
Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.0/58.0 kB 928.8 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 1.8 MB/s eta 0:00:00 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [11]:
pip install huggingface-hub

Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install wandb

Note: you may need to restart the kernel to use updated packages.


# **CONFIGURING THE WANDB & HUGGINGFACE HUB**

In [40]:
from huggingface_hub import notebook_login

!notebook_login()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/bin/bash: -c: line 1: syntax error: unexpected end of file


In [ ]:
wandb login

# **IMPORTING THE NECESSARY LIBRARIES**

In [13]:
import os
import evaluate
import numpy as np
import pandas as pd
from datasets import load_dataset, Dataset, DatasetDict
from transformers import AutoTokenizer, Seq2SeqTrainingArguments, Seq2SeqTrainer,DataCollatorForSeq2Seq, AutoModelForSeq2SeqLM

2024-04-12 22:30:38.571636: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-12 22:30:38.571761: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-12 22:30:38.842279: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Initialising wandb

In [14]:
import wandb

# **PIPELINE**

In [15]:
dataset = load_dataset("wmt14", 'de-en')

Generating train split:   0%|          | 0/4508785 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3003 [00:00<?, ? examples/s]

In [16]:
dataset

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 4508785
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 3000
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 3003
    })
})

In [17]:
new_dataset = dataset["train"].select(range(150000))

In [18]:
new_dataset

Dataset({
    features: ['translation'],
    num_rows: 150000
})

In [19]:
total_samples = 150000
train_samples = int(total_samples * 0.7)
valid_samples = int(total_samples * 0.1)
test_samples = total_samples - train_samples - valid_samples

In [20]:
train_dataset = new_dataset.select(range(train_samples))
valid_dataset = new_dataset.select(range(train_samples, train_samples + valid_samples))
test_dataset = new_dataset.select(range(train_samples + valid_samples, total_samples))

In [21]:
train_dataset.to_csv("/kaggle/working/train_dataset.csv", index=False)

valid_dataset.to_csv("/kaggle/working/valid_dataset.csv", index=False)

test_dataset.to_csv("/kaggle/working/test_dataset.csv", index=False)

Creating CSV from Arrow format:   0%|          | 0/105 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/15 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/30 [00:00<?, ?ba/s]

10158133

In [21]:
df_1 = pd.read_csv("/kaggle/working/train_dataset.csv")
df_2 = pd.read_csv("/kaggle/working/valid_dataset.csv")
df_3 = pd.read_csv("/kaggle/working/test_dataset.csv")

df_1['en_length'] = df_1['translation'].apply(lambda x: len(eval(x)['en']))
df_2['en_length'] = df_2['translation'].apply(lambda x: len(eval(x)['en']))
df_3['en_length'] = df_3['translation'].apply(lambda x: len(eval(x)['en']))

df_sorted_1 = df_1.sort_values(by='en_length')
df_sorted_2 = df_2.sort_values(by='en_length')
df_sorted_3 = df_3.sort_values(by='en_length')

df_sorted_1.drop(columns='en_length', inplace=True)
df_sorted_2.drop(columns='en_length', inplace=True)
df_sorted_3.drop(columns='en_length', inplace=True)

df_sorted_1.to_csv("/kaggle/working/en_to_de_train_dataset.csv", index=False)
df_sorted_2.to_csv("/kaggle/working/en_to_de_valid_dataset.csv", index=False)
df_sorted_3.to_csv("/kaggle/working/en_to_de_test_dataset.csv", index=False)

In [22]:
train_df = pd.read_csv("/kaggle/working/en_to_de_train_dataset.csv")
valid_df = pd.read_csv("/kaggle/working/en_to_de_valid_dataset.csv")
test_df = pd.read_csv("/kaggle/working/en_to_de_test_dataset.csv")

train_dataset = Dataset.from_pandas(train_df)
valid_dataset = Dataset.from_pandas(valid_df)
test_dataset = Dataset.from_pandas(test_df)

In [22]:
checkpoint = "google-t5/t5-base"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5_fast.py:171: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on google-t5/t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


# **English to German Translation**

In [24]:
train_dataset[150]

{'translation': "{'de': 'Abstimmungen', 'en': 'Vote'}"}

In [25]:
source_lang = "en"
target_lang = "de"
prefix = "translate English to German: "

def preprocess_function(example):
    if isinstance(example["translation"], str):
        example["translation"] = eval(example["translation"])
    
    input_text = example["translation"][source_lang]
    target_text = example["translation"][target_lang]
    
    # Tokenize input and target text with max_length=128
    inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True, max_length=128)
    targets = tokenizer(target_text, return_tensors="pt", padding=True, truncation=True, max_length=128)
    
    # Prepare model inputs and labels
    model_inputs = {
        "input_ids": inputs.input_ids.flatten(),
        "attention_mask": inputs.attention_mask.flatten(),
        "labels": targets.input_ids.flatten()
    }
    
    return model_inputs

In [26]:
tokenized_train_example = preprocess_function(train_dataset[150])

tokenized_valid_example = preprocess_function(valid_dataset[150])

tokenized_test_example = preprocess_function(test_dataset[150])

In [27]:
tokenized_train = [preprocess_function(example) for example in train_dataset]
tokenized_valid = [preprocess_function(example) for example in valid_dataset]
tokenized_test = [preprocess_function(example) for example in test_dataset]

In [28]:
tokenized_train_example

{'input_ids': tensor([3152,   17,   15,    1]),
 'attention_mask': tensor([1, 1, 1, 1]),
 'labels': tensor([  891, 11822,    35,     1])}

In [29]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [30]:
metric = evaluate.load("sacrebleu")

In [31]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [32]:
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [33]:
model

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [34]:
training_args = Seq2SeqTrainingArguments(
    output_dir = "./model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    num_train_epochs=1,
    report_to="wandb",
    predict_with_generate=True,
    fp16=True,
)

In [35]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_valid,
    tokenizer=tokenizer,
    data_collator = data_collator,
    compute_metrics=compute_metrics,
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [36]:
trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,1.081100,0.932104,8.340600,18.002200


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

TrainOutput(global_step=3282, training_loss=1.1024056015619048, metrics={'train_runtime': 3644.8688, 'train_samples_per_second': 28.808, 'train_steps_per_second': 0.9, 'total_flos': 1.081040125759488e+16, 'train_loss': 1.1024056015619048, 'epoch': 1.0})

In [37]:
wandb.finish()

eval/bleu,▁
eval/gen_len,▁
eval/loss,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/epoch,▁▂▄▅▆▇██
train/global_step,▁▂▄▅▆▇██
train/grad_norm,▁█▄▅▄▇
train/learning_rate,█▇▅▄▂▁
train/loss,█▃▂▂▁▂


In [48]:
from huggingface_hub import notebook_login
notebook_login()

In [49]:
trainer.push_to_hub("Reyansh4/NMT_T5_wmt14_en_to_de")

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.05k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Reyansh4/model/commit/ee5a16cb5758cfc81e71710577259ee5bb2ebf3d', commit_message='Reyansh4/NMT_T5_wmt14_en_to_de', commit_description='', oid='ee5a16cb5758cfc81e71710577259ee5bb2ebf3d', pr_url=None, pr_revision=None, pr_num=None)

In [50]:
tokenizer.push_to_hub("Reyansh4/NMT_T5_wmt14_en_to_de")

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Reyansh4/NMT_T5_wmt14_en_to_de/commit/c6dfbace323c4444c1d9ad2b25eff779b6b159fb', commit_message='Upload tokenizer', commit_description='', oid='c6dfbace323c4444c1d9ad2b25eff779b6b159fb', pr_url=None, pr_revision=None, pr_num=None)

In [55]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("/kaggle/working/model/checkpoint-3000")

# Load the model
model = AutoModelForSeq2SeqLM.from_pretrained("/kaggle/working/model/checkpoint-3000")

input_text = "this is something very good"

inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)
outputs = model.generate(input_ids=inputs.input_ids, attention_mask=inputs.attention_mask)
translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Translated Text:", translated_text)

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Translated Text: Das ist etwas sehr Gutes.


In [57]:
tokenizer = AutoTokenizer.from_pretrained("/kaggle/working/model/checkpoint-3000")
model = AutoModelForSeq2SeqLM.from_pretrained("/kaggle/working/model/checkpoint-3000")

In [ ]:
import gradio as gr

gr.Interface(
    fn=translate,
    inputs=gr.Textbox(lines=5, label="Input Text"),
    outputs="Translated Query",
    title="Translation from English to German",
).launch(share=True)

# **German to English Translation**

In [23]:
df_1 = pd.read_csv("/kaggle/working/train_dataset.csv")
df_2 = pd.read_csv("/kaggle/working/valid_dataset.csv")
df_3 = pd.read_csv("/kaggle/working/test_dataset.csv")

df_1['de_length'] = df_1['translation'].apply(lambda x: len(eval(x)['de']))
df_2['de_length'] = df_2['translation'].apply(lambda x: len(eval(x)['de']))
df_3['de_length'] = df_3['translation'].apply(lambda x: len(eval(x)['de']))

df_sorted_1 = df_1.sort_values(by='de_length')
df_sorted_2 = df_2.sort_values(by='de_length')
df_sorted_3 = df_3.sort_values(by='de_length')

df_sorted_1.drop(columns='de_length', inplace=True)
df_sorted_2.drop(columns='de_length', inplace=True)
df_sorted_3.drop(columns='de_length', inplace=True)

df_sorted_1.to_csv("/kaggle/working/de_to_en_train_dataset.csv", index=False)
df_sorted_2.to_csv("/kaggle/working/de_to_en_valid_dataset.csv", index=False)
df_sorted_3.to_csv("/kaggle/working/de_to_en_test_dataset.csv", index=False)

In [24]:
train_df = pd.read_csv("/kaggle/working/de_to_en_train_dataset.csv")
valid_df = pd.read_csv("/kaggle/working/de_to_en_valid_dataset.csv")
test_df = pd.read_csv("/kaggle/working/de_to_en_test_dataset.csv")

train_dataset = Dataset.from_pandas(train_df)
valid_dataset = Dataset.from_pandas(valid_df)
test_dataset = Dataset.from_pandas(test_df)

In [25]:
train_dataset[150]

{'translation': "{'de': 'Weshalb?', 'en': 'Why?'}"}

In [26]:
source_lang = "de"
target_lang = "en"
prefix = "translate German to English: "

def preprocess_function(example):
    if isinstance(example["translation"], str):
        example["translation"] = eval(example["translation"])
    
    input_text = example["translation"][source_lang]
    target_text = example["translation"][target_lang]
    
    # Tokenize input and target text with max_length=128
    inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True, max_length=128)
    targets = tokenizer(target_text, return_tensors="pt", padding=True, truncation=True, max_length=128)
    
    # Prepare model inputs and labels
    model_inputs = {
        "input_ids": inputs.input_ids.flatten(),
        "attention_mask": inputs.attention_mask.flatten(),
        "labels": targets.input_ids.flatten()
    }
    
    return model_inputs

In [27]:
tokenized_train_example = preprocess_function(train_dataset[150])

tokenized_valid_example = preprocess_function(valid_dataset[150])

tokenized_test_example = preprocess_function(test_dataset[150])

In [28]:
tokenized_train = [preprocess_function(example) for example in train_dataset]
tokenized_valid = [preprocess_function(example) for example in valid_dataset]
tokenized_test = [preprocess_function(example) for example in test_dataset]

In [29]:
tokenized_train_example

{'input_ids': tensor([  101, 14737,    58,     1]),
 'attention_mask': tensor([1, 1, 1, 1]),
 'labels': tensor([1615,   58,    1])}

In [30]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [31]:
metric = evaluate.load("sacrebleu")

In [32]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [33]:
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [34]:
model

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [35]:
training_args = Seq2SeqTrainingArguments(
    output_dir = "./model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    num_train_epochs=1,
    predict_with_generate=True,
    report_to="wandb",
    fp16=True,
)

In [36]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_valid,
    tokenizer=tokenizer,
    data_collator = data_collator,
    compute_metrics=compute_metrics,
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [37]:
trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,1.812800,1.568120,10.498300,17.378900


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

TrainOutput(global_step=3282, training_loss=1.8945422500898603, metrics={'train_runtime': 3553.0277, 'train_samples_per_second': 29.552, 'train_steps_per_second': 0.924, 'total_flos': 1.365896354512896e+16, 'train_loss': 1.8945422500898603, 'epoch': 1.0})

In [45]:
from huggingface_hub import notebook_login
notebook_login()

In [46]:
trainer.push_to_hub("Reyansh4/NMT_T5_wmt14_de_to_en")

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.05k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/Reyansh4/model/commit/2d14a58386c1a9568fbdefa8ca20e17d1e0731dc', commit_message='Reyansh4/NMT_T5_wmt14_de_to_en', commit_description='', oid='2d14a58386c1a9568fbdefa8ca20e17d1e0731dc', pr_url=None, pr_revision=None, pr_num=None)

In [47]:
tokenizer.push_to_hub("Reyansh4/NMT_T5_wmt14_de_to_en")

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Reyansh4/NMT_T5_wmt14_de_to_en/commit/f95540f9426712416ac8b2eb3ddf6ab91de4c818', commit_message='Upload tokenizer', commit_description='', oid='f95540f9426712416ac8b2eb3ddf6ab91de4c818', pr_url=None, pr_revision=None, pr_num=None)

In [48]:
wandb.finish()

eval/bleu,▁
eval/gen_len,▁
eval/loss,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/epoch,▁▂▄▅▆▇██
train/global_step,▁▂▄▅▆▇██
train/grad_norm,█▅▁█▂▂
train/learning_rate,█▇▅▄▂▁
train/loss,█▄▂▂▁▁


In [49]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("/kaggle/working/model/checkpoint-3000")

# Load the model
model = AutoModelForSeq2SeqLM.from_pretrained("/kaggle/working/model/checkpoint-3000")

input_text = "Das ist etwas sehr Gutes"

inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)
outputs = model.generate(input_ids=inputs.input_ids, attention_mask=inputs.attention_mask)
translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Translated Text:", translated_text)

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Translated Text: That is something very good.


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Reyansh4/NMT_T5_wmt14_de_to_en")
model = AutoModelForSequenceClassification.from_pretrained("Reyansh4/NMT_T5_wmt14_de_to_en")

In [ ]:
import gradio as gr

gr.Interface(
    fn=translate,
    inputs=gr.Textbox(lines=5, label="Input Text"),
    outputs="Translated Query",
    title="Translation from German to English",
).launch(share=True)